# 多人模式貪食蛇

剛剛我們一起做完貪食蛇遊戲了，這次我們大家連線一起玩貪食蛇吧！

### 遊戲規則

- 這次我們的貪食蛇是多人模式，每個人控制一條蛇，蛇的顏色不同
- 遊戲開始時，每條蛇都在地圖的隨機位置，並且有一個隨機的方向
- 每條蛇的目標是吃到地圖上的食物，吃到食物後蛇的身體會變長
- 當蛇的頭碰到地圖的邊界或者碰到任何蛇的身體時，蛇會死亡
- 你需要透過程式碼寫出你的蛇的移動策略，嘗試讓你的蛇吃到最多的食物！

### 連線方式

- 執行下面的程式碼，輸入你的名字，你就成功連線並開始和別的玩家比賽了！
- 然後[點我](https://youmingyeh.github.io/multiplayer-snake-game-websocket/)進入這個網址或直接在這裡就可以看到你的蛇囉！

**_現在，請修改以下 `change_direction()` 函數，寫出屬於你的蛇的移動策略！_**


In [1]:
# !python -m pip install python-socketio nest_asyncio asyncio ipython -U --force-reinstall

## 函式介紹
### 什麼是函式 function？
```python
def function(a): # 傳入參數
    # 做一些事情...
    a = a + 1
    return a # 結果

# 呼叫函示
b = function(a)
```

>也就是說，函式是一段預先寫好的程式碼，它可以接收一些參數，做一些事情，然後可能會返回一個結果。

In [2]:
import asyncio
import socketio
import random
import json

sio = socketio.AsyncClient()
map = []
snakes = {}
foods = []
player_id = None
FRAME_RATE = 0.1
score = 0

def direction_to_string(direction):
    if direction == [0, -1]:
        return "up"
    if direction == [0, 1]:
        return "down"
    if direction == [-1, 0]:
        return "left"
    if direction == [1, 0]:
        return "right"
    return None

@sio.event
async def connect():
    print("Connected to the server")
    await sio.emit("register", {"player_id": player_id})
    
@sio.event
async def disconnect():
    global score
    print("你死掉了！")
    print(f"你最後的分數是：{score}")
    
@sio.event
async def state(data):
    global map, snakes, foods, player_id
    data = json.loads(data)
    map = data["map"]
    snakes = data["snakes_pos"]
    foods = data["foods_pos"]

    if player_id is None:
        return

    if str(player_id) not in snakes.keys():
        await sio.emit("exit")
        await sio.disconnect()
        return
    
@sio.event
async def register(data):
    global player_id
    player_id = data["player_id"]
    print(f"成功註冊玩家！你的 ID 是： {player_id}")
    asyncio.create_task(send_moves())


async def send_moves():
    while True:
        global map, snakes, player_id, foods, score

        player_position = None
        player_direction = None
        if str(player_id) in snakes.keys():
            player_position = snakes[str(player_id)]["body"][0]
            player_direction = direction_to_string(snakes[str(player_id)]["direction"])

        if player_position is None or player_direction is None:
            await asyncio.sleep(FRAME_RATE)
            continue

        direction = choose_direction(map, player_position, player_direction)

        score = len(snakes[str(player_id)]["body"])

        message = {"type": "move", "direction": direction}
        await sio.emit("move", message)
        await asyncio.sleep(FRAME_RATE)


## 開始遊戲

In [ ]:
"""範例 1"""
def example1(map, player_position, player_direction):
    # 隨機方向
    directions = ["up", "down", "left", "right"]
    direction = random.choice(directions)
    return direction

"""範例 2"""
def example2(map, player_position, player_direction):
    directions = []
    # 遇到牆壁或蛇就轉彎
    if (
        player_position[0] > 0
        and (
            map[player_position[0] - 1][player_position[1]] == ""
            or map[player_position[0] - 1][player_position[1]] == "food"
        )
        and player_direction != "right"
    ):
        directions.append("left")
    if (
        player_position[0] < len(map) - 1
        and (
            map[player_position[0] + 1][player_position[1]] == ""
            or map[player_position[0] + 1][player_position[1]] == "food"
        )
        and player_direction != "left"
    ):
        directions.append("right")
    if (
        player_position[1] > 0
        and (
            map[player_position[0]][player_position[1] - 1] == ""
            or map[player_position[0]][player_position[1] - 1] == "food"
        )
        and player_direction != "down"
    ):
        directions.append("up")
    if (
        player_position[1] < len(map[0]) - 1
        and (
            map[player_position[0]][player_position[1] + 1] == ""
            or map[player_position[0]][player_position[1] + 1] == "food"
        )
        and player_direction != "up"
    ):
        directions.append("down")

    direction = random.choice(directions)
    return direction

"""範例 3"""
def example3(map, player_position, player_direction):
    directions = {'up': 0, 'down': 0, 'left': 0, 'right': 0}
    max_row = len(map) 
    max_col = len(map[0]) 
    x, y = player_position

    # 評估每個方向上的食物數量和安全性
    for dx, dy, dir in [(-1, 0, 'left'), (1, 0, 'right'), (0, -1, 'up'), (0, 1, 'down')]:
        distance = 1  # Start checking from the next cell in the direction
        while True:
            nx, ny = x + dx * distance, y + dy * distance
            if 0 <= nx < max_row and 0 <= ny < max_col:
                if map[nx][ny] == "food":
                    directions[dir] += 20 - distance # 如果這方向有食物，就 +（20 - 和食物的距離） 權重 -> 自己調整看看吧
                if map[nx][ny] == "":
                    directions[dir] += 0 # 如果這方向有空位，就 +0 權重 -> 自己調整看看吧
                else:
                    directions[dir] += -5 # 如果這方向有蛇，就 -1 權重 -> 自己調整看看吧
            else:
                break
            distance += 1

        nx, ny = x + dx * 1, y + dy * 1
        if (0 > nx or nx >= max_row) or (0 > ny or ny >= max_col):
            directions[dir] += -100 
        elif map[nx][ny] != "food" and map[nx][ny] != "":
            directions[dir] += -100
            
    # 不能走反方向
    directions[opposite_direction(player_direction)] = -1000
    
    # 過濾出得分最高的方向
    max_score = max(directions.values())
    best_directions = [dir for dir, score in directions.items() if score == max_score]

    # 如果當前方向在最佳方向中且不需要轉向，則優先保持當前方向
    if player_direction in best_directions:
        return player_direction

    # 從最佳方向中隨機選擇一個
    return random.choice(best_directions)

def opposite_direction(direction):
    """找到相反方向避免回頭"""
    if direction == "up":
        return "down"
    elif direction == "down":
        return "up"
    elif direction == "left":
        return "right"
    elif direction == "right":
        return "left"

"""你的策略"""
def my_plan(map, player_position, player_direction):
    # TODO: 完成屬於你的策略
    return "up"

"""選擇方向"""
def choose_direction(map, player_position, player_direction):
    ###########################################################
    # TODO: 完成你移動蛇的策略
    # 現在
    # direction: 你想要蛇的移動方向，可以是 "up", "down", "left", "right" 之一
    # player_position: 你的蛇的頭的位置
    # player_direction: 你的蛇的目前移動方向
    # map: 目前地圖上所有的資訊
    # map[i][j] = "" 代表地圖上(i, j)位置是空的
    # map[i][j] = "food" 代表地圖上(i, j)位置有食物
    # map[i][j] = "player" 代表地圖上(i, j)位置有其他玩家的蛇
    ###########################################################
    # 選擇你的策略！策略的函式程式碼在上面喔，你可以使用我們幫你寫好的，也可以自己寫一個！
    # 範例 1：隨機移動
    # return example1(map, player_position, player_direction)
    
    # 範例 2：非常害怕撞到東西，會一直往可以躲開牆壁、躲開蛇的方向移動
    # return example2(map, player_position, player_direction)

    # 範例 3: 非常喜歡往有食物的方向移動，遇到牆壁或蛇則會想辦法轉彎，但有時候會為了找食物就不小心撞牆了...
    # return example3(map, player_position, player_direction)

    # 你自己的策略：寫一個屬於你的策略，請修改 def my_plan() 裡的程式碼
    # return my_plan(map, player_position, player_direction)
    ###########################################################
    # 呼叫你的函式
    # 這是範例 1，你可以改成範例 2、3，或寫一個屬於你自己的版本
    
    return example3(map, player_position, player_direction) 
    ###########################################################


from IPython.display import IFrame

async def main():
    global player_id
    player_id = input("輸入你的遊戲 ID: ")
    iframe = IFrame("http://140.112.106.45:3000/", 900,500)
    display(iframe)
    await sio.connect("http://140.112.106.45:3000/")
    await sio.wait()
import nest_asyncio
nest_asyncio.apply()

await main()

輸入你的遊戲 ID:  yym


Connected to the server
成功註冊玩家！你的 ID 是： yym
